# **Лабораторная работа 3. Методы поиска подстроки в строке.**

## **Выполнил студент группы БВТ 1903 Шевченко А.А.**

###                                                        Задание 1

Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

Алгоритмы:

- Кнута-Морриса-Пратта
  
- Упрощенный Бойера-Мура

In [ ]:
print('Введите строку для поиска и сроку, которую нужно искать')
main_string =input()
string_to_find = input()
is_case_sensitive = False


print('должен ли поиск быть чувствительный к регистру введите yes, если да, если нет что либо другое')
buffer = input()
if buffer == 'yes':
    is_case_sensitive = True

def prefix(s):
    str_prefix = [0]*len(s)
    for i in range(1, len(s)):
        k = str_prefix[i-1]
        while k > 0 and s[k] != s[i]:
            k = str_prefix[k-1]
        if s[k] == s[i]:
            k = k + 1
        str_prefix[i] = k
    return str_prefix

# Кнута-Морриса-Пратта
def kmp(s_main, s_sought, case_sensitive=True):
    if not case_sensitive:
        s_main = s_main.lower()
        s_sought = s_sought.lower()

    index = -1
    str_prefix = prefix(s_main)
    k = 0
    for i in range(len(s_sought)):
        while k > 0 and s_main[k] != s_sought[i]:
            k = str_prefix[k-1]
        if s_main[k] == s_sought[i]:
            k = k + 1
        if k == len(s_main):
            index = i - len(s_main) + 1
            break
    return index

def build_table(s):
    dic_table = dict()
    length = len(s)
    for i in range(length - 1):
        dic_table[s[i]] = length - i - 1
    if not s[length - 1] in dic_table:
        dic_table[s[length - 1]] = length
    return dic_table

#Упрощенный Бойера-Мура
def boyer_mur_search(s_main, s_sought, case_sensitive=True):
    if not case_sensitive:
        s_main = s_main.lower()
        s_sought = s_sought.lower()

    dic_table = build_table(s_sought)
    s_sought_len = len(s_sought)
    current_step = s_sought_len - 1

    while current_step < len(s_main):  # основной цикл для поиска подстроки

        s_sought_i = s_sought_len - 1  
        for i in range(current_step, current_step - s_sought_len, -1):
            if s_main[i] != s_sought[s_sought_i]:  
                if s_main[i] in dic_table:
                    step = dic_table[s_main[i]]
                else:
                    step = s_sought_len
                break

            if i == current_step - s_sought_len + 1:  
                return i
            s_sought_i -= 1

        current_step += step  # двигаем место поиска на шаг
print(kmp(string_to_find, main_string, is_case_sensitive))
print(kmp(string_to_find, main_string, False))

print(boyer_mur_search(main_string, string_to_find, case_sensitive=is_case_sensitive))
print(boyer_mur_search(main_string, string_to_find, case_sensitive=False))

### Задание 2
**Задача:**

написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.

**Входные данные:**

массив чисел, представляющий собой расстановку в
порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.

**Выходные данные:** 

если решения нет, то функция должна вернуть
пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента.

Например, для начального расположения элементов [1, 2, 3, 4, 5, 6, 7, 8,
13, 9, 11, 12, 10, 14, 15, 0] одним из возможных решений будет [15, 14, 10, 13,
9, 10, 14, 15] (последовательность шагов здесь: двигаем 15, двигаем 14,
двигаем 10, …, двигаем 15).

In [ ]:
from queue import PriorityQueue

N = 4

# Функция, определяющая движение пятнашек
def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # движение вниз
    if i < N - 1: offsets.append(N)  # движение вверх
    if j > 0: offsets.append(-1)     # движение вправо
    if j < N - 1: offsets.append(1)  # движение влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))

# Функция, определяющая есть ли у пятнашек конечное решение
def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


class Position:
    # Функция, принимающая позицию и начальную дистанцию
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    # Функция, срабатывающая при сравнении (<) объекта с другим объектом
    def __lt__(self, other):
        return self.start_distance < other.start_distance

    # Функция, срабатывающая при использовании объекта как строки
    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) 
for i in range(0, N*N, N))

SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

# Проверяем, можно ли в данной расстановке найти решение. Если нет, то выводим предупреждающее сообщение
if parity(start) == 0:
    print('Задачу решить нельзя')
else:
    start = tuple(start)

    p = Position(start, 0)

    print(p)
    print()

    # Кладем в очередь с приоритетом первоначальную позицию
    candidates = PriorityQueue()
    candidates.put(p)

    # Кортеж посещенных нами позиций
    visited = set([p])

    came_from = {p.position: None}
    
    while p.position != SOLVED:
        # Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # Кладем в очередь все соседние позиции. Повторяем не вытащим конечную позицию из очереди
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    # Последовательное решение пятнашек (путь)
    path = []
    # Сохраняем конечную позицию
    prev = p
    # Идем в обратном порядке и запоминаем очередность хода в path
    while p.position != start:
        # Запоминаем откуда ход
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

## Вывод: В ходе данной лабораторной работы я познакомился и реализовал два алгоритма поиска: Кнута-Морриса-Пратта и Упрощенный Бойера-Мура. Так же написал алгоритм для игры в пятнашки.